In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)
    symbol_files[symbol] = f
symbol_files
#allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

{'1000XECUSDT': 'cfgs_live\\1000XEC.json',
 '100SHIBUSDT': 'cfgs_live\\100SHIB.json',
 '1INCHUSDT': 'cfgs_live\\1INCH.json',
 'ADAUSDT': 'cfgs_live\\ADA.json',
 'AKROUSDT': 'cfgs_live\\AKRO.json',
 'ALGOUSDT': 'cfgs_live\\ALGO.json',
 'ALICEUSDT': 'cfgs_live\\ALICE.json',
 'ALPHAUSDT': 'cfgs_live\\ALPHA.json',
 'ANKRUSDT': 'cfgs_live\\ANKR.json',
 'API3USDT': 'cfgs_live\\API3.json',
 'ARUSDT': 'cfgs_live\\AR.json',
 'ATOMUSDT': 'cfgs_live\\ATOM.json',
 'BAKEUSDT': 'cfgs_live\\BAKE.json',
 'BANDUSDT': 'cfgs_live\\BAND.json',
 'BELUSDT': 'cfgs_live\\BEL.json',
 'BLZUSDT': 'cfgs_live\\BLZ.json',
 'BNBUSDT': 'cfgs_live\\BNB.json',
 'BTSUSDT': 'cfgs_live\\BTS.json',
 'C98USDT': 'cfgs_live\\C98.json',
 'CHZUSDT': 'cfgs_live\\CHZ.json',
 'COMPUSDT': 'cfgs_live\\COMP.json',
 'COTIUSDT': 'cfgs_live\\COTI.json',
 'CRVUSDT': 'cfgs_live\\CRV.json',
 'DENTUSDT': 'cfgs_live\\DENT.json',
 'DGBUSDT': 'cfgs_live\\DGB.json',
 'DODOUSDT': 'cfgs_live\\DODO.json',
 'DOGEUSDT': 'cfgs_live\\DOGE.json',
 'DOT

In [4]:
## NOW copy result folders :)
import json
import shutil

files = glob.iglob('backtests/binance/' + '**/live_config.json', recursive=True)

for filename in files:
    with open(filename) as f:
        config = json.load(f)
    symbol = config['config_name'].split('_')[2]

    if symbol not in symbol_files: continue

    with open(symbol_files[symbol]) as f:
        production_config = json.load(f)

    long_pc = production_config['long']
    long_c = config['long']

    k_pc = long_pc.keys()
    k_c = long_c.keys()

    if len(k_c) != len(k_c): continue

    same = True
    for k in k_pc:
        if k not in k_c:
            same = False
            break
        v_pc = long_pc[k]
        v_c = long_c[k]
        #print(f'{k}={v_c},{v_pc}')

    if not same: continue

    print(f'{filename} is same as {symbol_files[symbol]}')

    src = filename.replace('\\live_config.json', '')
    dst = f'backtests/binance_production/{symbol}'
    shutil.copytree(src=src, dst=dst, dirs_exist_ok=True)



backtests/binance\ADAUSDT\plots\2022-04-03T202719\live_config.json is same as cfgs_live\ADA.json
backtests/binance\ALICEUSDT\plots\2022-04-05T084248\live_config.json is same as cfgs_live\ALICE.json
backtests/binance\API3USDT\plots\2022-04-05T085246\live_config.json is same as cfgs_live\API3.json
backtests/binance\ATOMUSDT\plots\2022-04-04T110127\live_config.json is same as cfgs_live\ATOM.json
backtests/binance\BNBUSDT\plots\2022-04-04T105245\live_config.json is same as cfgs_live\BNB.json
backtests/binance\CHZUSDT\plots\2022-04-03T203915\live_config.json is same as cfgs_live\CHZ.json
backtests/binance\CRVUSDT\plots\2022-04-06T005705\live_config.json is same as cfgs_live\CRV.json
backtests/binance\DOGEUSDT\plots\2022-04-04T230406\live_config.json is same as cfgs_live\DOGE.json
backtests/binance\DOTUSDT\plots\2022-04-03T085136\live_config.json is same as cfgs_live\DOT.json
backtests/binance\DYDXUSDT\plots\2022-04-06T005530\live_config.json is same as cfgs_live\DYDX.json
backtests/binance\

In [5]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance_production','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_614140/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_614140/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_614140/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_614140/891417201.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\ADAUSDT\backtest_result.txt,5.5,ADAUSDT,True,15.76,0.0502,100.0,0.8776,True,37.180,0.11100,100.00,0.8613
1,\ALICEUSDT\backtest_result.txt,5.5,ALICEUSDT,True,21.72,0.0850,100.0,0.8017,True,5.283,0.03270,100.00,0.7321
2,\API3USDT\backtest_result.txt,5.5,API3USDT,True,16.46,0.3490,100.0,0.9537,True,18.290,0.46700,100.00,0.8913
3,\ATOMUSDT\backtest_result.txt,5.5,ATOMUSDT,True,37.01,0.1280,100.0,0.9440,True,14.840,0.07030,100.00,0.7710
4,\BNBUSDT\backtest_result.txt,5.5,BNBUSDT,True,20.32,0.0738,100.0,0.9283,True,9.426,0.02060,100.00,0.8129
5,\CHZUSDT\backtest_result.txt,5.5,CHZUSDT,True,39.34,0.1310,100.0,0.8653,True,26.910,0.07480,100.00,0.7790
6,\CRVUSDT\backtest_result.txt,5.5,CRVUSDT,True,39.52,0.1360,100.0,0.9182,True,0.000,0.10400,33.99,0.3125
7,\DOGEUSDT\backtest_result.txt,5.5,DOGEUSDT,True,27.02,0.0977,100.0,0.8941,True,48.100,0.15600,100.00,0.8776
8,\DOTUSDT\backtest_result.txt,5.5,DOTUSDT,True,36.94,0.1290,100.0,0.8642,True,24.320,0.06700,100.00,0.8015
9,\DYDXUSDT\backtest_result.txt,5.5,DYDXUSDT,True,31.13,0.1280,100.0,0.9144,True,43.290,0.16700,100.00,0.7776


Now filter the dataframe

In [6]:
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              # & (big_data['adgt_%'] > 0.1)
              # & (big_data['adgl_%'] > 0.04)
              # & (big_data['adgs_%'] > 0.04)
              # & (big_data['bnkrl_%'] > 99.0)
              # & (big_data['bnkrs_%'] > 99.0)
              # & (big_data['ebrs'] > .9)
              # #& (big_data['ebrl'] > .9)
              # & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
0,\ADAUSDT\backtest_result.txt,5.5,ADAUSDT,True,15.76,0.0502,100.0,0.8776,True,37.180,0.11100,100.00,0.8613,0.16120
1,\ALICEUSDT\backtest_result.txt,5.5,ALICEUSDT,True,21.72,0.0850,100.0,0.8017,True,5.283,0.03270,100.00,0.7321,0.11770
2,\API3USDT\backtest_result.txt,5.5,API3USDT,True,16.46,0.3490,100.0,0.9537,True,18.290,0.46700,100.00,0.8913,0.81600
3,\ATOMUSDT\backtest_result.txt,5.5,ATOMUSDT,True,37.01,0.1280,100.0,0.9440,True,14.840,0.07030,100.00,0.7710,0.19830
4,\BNBUSDT\backtest_result.txt,5.5,BNBUSDT,True,20.32,0.0738,100.0,0.9283,True,9.426,0.02060,100.00,0.8129,0.09440
5,\CHZUSDT\backtest_result.txt,5.5,CHZUSDT,True,39.34,0.1310,100.0,0.8653,True,26.910,0.07480,100.00,0.7790,0.20580
6,\CRVUSDT\backtest_result.txt,5.5,CRVUSDT,True,39.52,0.1360,100.0,0.9182,True,0.000,0.10400,33.99,0.3125,0.24000
7,\DOGEUSDT\backtest_result.txt,5.5,DOGEUSDT,True,27.02,0.0977,100.0,0.8941,True,48.100,0.15600,100.00,0.8776,0.25370
8,\DOTUSDT\backtest_result.txt,5.5,DOTUSDT,True,36.94,0.1290,100.0,0.8642,True,24.320,0.06700,100.00,0.8015,0.19600
9,\DYDXUSDT\backtest_result.txt,5.5,DYDXUSDT,True,31.13,0.1280,100.0,0.9144,True,43.290,0.16700,100.00,0.7776,0.29500


In [7]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
0,\ADAUSDT\backtest_result.txt,ADAUSDT,0.16120,0.0502,0.11100,100.0,100.00,0.8776,0.8613
1,\ALICEUSDT\backtest_result.txt,ALICEUSDT,0.11770,0.0850,0.03270,100.0,100.00,0.8017,0.7321
2,\API3USDT\backtest_result.txt,API3USDT,0.81600,0.3490,0.46700,100.0,100.00,0.9537,0.8913
3,\ATOMUSDT\backtest_result.txt,ATOMUSDT,0.19830,0.1280,0.07030,100.0,100.00,0.9440,0.7710
4,\BNBUSDT\backtest_result.txt,BNBUSDT,0.09440,0.0738,0.02060,100.0,100.00,0.9283,0.8129
5,\CHZUSDT\backtest_result.txt,CHZUSDT,0.20580,0.1310,0.07480,100.0,100.00,0.8653,0.7790
6,\CRVUSDT\backtest_result.txt,CRVUSDT,0.24000,0.1360,0.10400,100.0,33.99,0.9182,0.3125
7,\DOGEUSDT\backtest_result.txt,DOGEUSDT,0.25370,0.0977,0.15600,100.0,100.00,0.8941,0.8776
8,\DOTUSDT\backtest_result.txt,DOTUSDT,0.19600,0.1290,0.06700,100.0,100.00,0.8642,0.8015
9,\DYDXUSDT\backtest_result.txt,DYDXUSDT,0.29500,0.1280,0.16700,100.0,100.00,0.9144,0.7776
